<a href="https://www.kaggle.com/code/viniciusnalasantos/k-means-clustering?scriptVersionId=124367579" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Clustering: Extracting Patterns From Data & Concept + Feature Scaling + PCA [Pt.1]
## Customers segmentation based on their credit card usage behavior

![credit_cards.png](https://miro.medium.com/v2/resize:fit:828/0*S9zDFD_KE5yEiiaY)

This is the first article of a series in which I will outline an end-to-end clustering project. We will begin with some initial concepts, going to dataset exploratory analysis, passing through data preprocessing, grouping customers into clusters, and at the end, we will perform an analysis of the clusters, plus marketing suggestions.

 It's lots of things to do, so let's start!

 ### Table of Contents

[Part One](#part_one)
* [Clustering Definition](#clustering_definition)
* [Supervisionized Learning x Unsupervisionized Learning](#supervised_unsupervised)
* [K-Means Clustering](#concept)
* [Dataset Dictionary](#data_dictionary)
* [Project Aim](#project_aim)
* [EDA (Exploratory Data Analysis)](#eda)
* [Feature Scaling](#feature_scaling)
* [PCA (Principal Component Analysis)](#pca)
* [Cluster Visualization with Plotly](#visualization)
* [Conclusion](#conclu)

[Part Two](#part_two)
* [Coming soon...](#)

<a id="part_one"></a>
<h2><b> Part One </b></h2>

---

#### Code from the article: [Clustering: Extracting Patterns From Data & Concept + Feature Scaling + PCA [Pt.1]](https://medium.com/@viniciusnala/clustering-extracting-patterns-from-data-concept-feature-scaling-pca-pt-1-4f33c6d39a71)

<a id="clustering_definition"></a>
## Clustering Definition

Clustering or data grouping analysis is a set of data mining techniques that aims to make an automatic grouping of the data according to their degree of similarity. The criterion of similarity depends on the problem and algorithm. The result of this process is the division of a data set into a certain amount of groups (clusters).

This is a very important definition, and all that we will do after that is related somewhat to this definition. Another very important definition is the difference between the two concepts below.

<a id="supervised_unsupervised"></a>
## Supervised Learning x Unsupervised Learning

Supervised Learning is an approach where an algorithm is trained on input data that has been labeled for a particular output. The model fits the data capturing the underlying pattern and relationship between the input data and the output labels, enabling it to predict outcomes accurately.

The two anterior data science projects that I did:

- [Machine Learning Model to Predict Survival in Titanic [Pt. 1]](https://medium.com/@viniciusnala/machine-learning-model-to-predict-survival-in-titanic-pt-1-b3681d1794fb)
- [Linear Regression: Advanced Modeling Techniques & Pipeline [Pt.1]](https://medium.com/@viniciusnala/linear-regression-advanced-modeling-techniques-pipeline-pt-1-3c0433230b88)

I built a supervised machine learning model, which I used to predict a "target variable" (label).

For example, in the first project, I constructed a model to predict if a person would survive or not on Titanic. To do that I provided data containing general characteristics of each passenger and the information if this person survived or not on Titanic (target variable) - the model captured the underlying pattern and relationship between the general characteristics of the passenger and if he survived or not -, then I presented to it neve-before-seen data with general characteristics of some passengers, but without the information if the passengers survived or not, being the model responsible to guess who would survive or not.

Unlike, supervised learning, unsupervised learning uses unlabeled data, i.e. we don't provide the target variable (label) when training the model. These algorithms aren't used to predict something but to discover hidden patterns in data without the need for human intervention (hence, they are "unsupervised").

If the difference is still not so clear, along the project you will understand better. Let's continue!

<a id="concept"></a>
## K-Means Clustering

The goal of clustering is to identify meaningful groups of data. These groups can be analyzed in depth, or be passed as a feature, or as an outcome to a classification or regression model. K-Means was the first clustering method to be developed, has a very simple algorithm, and is still widely used.

![clustering.png](https://miro.medium.com/v2/resize:fit:1100/format:webp/0*Cei6GQZkabRgn7J8.png)

K-Means divides the data into a K number of clusters, each cluster has a centroid. The main aim of this algorithm is to minimize the sum of the squared distances of each point to the centroid of its assigned cluster. K-Means does not ensure the clusters will have the same size but finds the clusters that are best separated.

Let's see a simple example, imagine that we have an N number of records and two variables X, and Y. Suppose we wanna split the data into K = 3 clusters, which means assigning each record (Xi, Yi) to a cluster K.

![cluster_1.png](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*xA_xdOKC9-GA0xcbw77fog.png)

The algorithm first selects random coordinates for the centroids, and using the euclidean distance formula measures the distance between each data point and each cluster centroid. Then each point is assigned to the closest centroid.

![distance.png](https://miro.medium.com/v2/resize:fit:600/format:webp/0*uvyEvT5qpoyv2cFX.png)

![cluster_2.png](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*ozKTU9kb_OgbU-QFA4myLA.png)

After that, the coordinates of the centroids are re-computed by taking the mean of all data points contained in that cluster. Given an assignment of Nk records to cluster K, the center of the cluster(Xk, Yk) is calculated through the equation:

![equation_1.png](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*ZCZ97gHVIh2j1FExJzXpow.png)

In simple terms, we are just summing the Xk and Yk of the cluster and dividing by the number of points in that cluster.

After this process, the algorithm computes the sum of squares within each cluster, which is given by the formula:

![equation_2.png](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*Te9NFMLZ9v5ZnIl_Zsp7Tg.png)

The K-Means keep repeating the process of measuring the distance between the points and the centroids, assigning each data point to the closest center, and re-computing the new coordinates of the centroids until the sum of squares across all three clusters is minimized.

![equation_3.png](https://miro.medium.com/v2/resize:fit:640/format:webp/1*odKVjftJNYpoZ9iJhudejA.png)

![cluster_3.png](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*t_cti7a3DasJ9BT1YpOTmg.png)

Repeat the same process many times can require a great computational cost, especially when the amount of data is large. 

If you wanna see by yourself how it works, I highly recommend this simulator: [K-Means Clustering Simulator](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/).

<a id="data_dictionary"></a>
## Dataset Dictionary

The dataset that will be used consists of the credit card usage behavior of 8950 customers during 6 months, having 18 behavioral features. The link to the dataset is available on Kaggle.

[Credit Card Dataset for Clustering](https://www.kaggle.com/datasets/arjunbhasin2013/ccdata)

- CUST_ID: Identification of Credit Card holder (Categorical)
- BALANCE: Balance amount left in their account to make purchases
- BALANCE_FREQUENCY: How frequently the Balance is updated, score between 0 and 1 (1 = frequently updated, 0 = not frequently updated)
- PURCHASES: Amount of purchases made from account
- ONEOFF_PURCHASES: Maximum purchase amount done in one-go
- INSTALLMENTS_PURCHASES: Amount of purchase done in installment
- CASH_ADVANCE: Cash in advance given by the user
- PURCHASES_FREQUENCY: How frequently the Purchases are being made, score between 0 and 1 (1 = frequently purchased, 0 = not frequently purchased)
- ONEOFFPURCHASESFREQUENCY : How frequently Purchases are happening in one-go (1 = frequently purchased, 0 = not frequently purchased)
- PURCHASESINSTALLMENTSFREQUENCY: How frequently purchases in installments are being done (1 = frequently done, 0 = not frequently done)
- CASHADVANCEFREQUENCY: How frequently the cash in advance being paid
- CASHADVANCETRX: Number of Transactions made with "Cash in Advanced"
- PURCHASES_TRX: Number of purchase transactions made
- CREDIT_LIMIT: Limit of Credit Card for user
- PAYMENTS: Amount of Payment done by user
- MINIMUM_PAYMENTS: Minimum amount of payments made by user
- PRCFULLPAYMENT: Percent of full payment paid by user
- TENURE: Tenure of credit card service for user

<a id="project_aim"></a>
## Project Aim

Initially, our focus will be to segment customers according to their similarities. After that, we will analyze this segmentation and define an effective credit card marketing strategy.

<a id="eda"></a>
## EDA (Exploratory Data Analysis)

Let's begin by importing the libraries and the dataset:

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/kaggle/input/ccdata/CC GENERAL.csv')

Visualizing some characteristics of the dataset:

In [ ]:
df.head(10)

In [ ]:
df.shape

Through Pandas Profiling we can see a very detailed report about the general characteristics of the data set.

In [ ]:
from pandas_profiling import ProfileReport

ProfileReport(df).to_notebook_iframe()

In [ ]:
# Dowload the report
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title="Pandas Profiling Report")
profile.to_file("report.html")

After analyzing the report I concluded that the features CUSTOMER_ID and TENURE are not going to contribute with the segmentation so we can exclude them from the dataset.

In [ ]:
# Delete the irrelevant features
df.drop(columns=['CUST_ID', 'TENURE'], inplace=True)

Now our dataset has 16 columns.

Let's see how many missing values there are in the dataset.

In [ ]:
# See Null Values
df.isnull().sum().sort_values(ascending=False)

Just a few ones, to deal with them I will use KNNImputer: each missing value will be imputed using the mean value from the n_neighbors nearest neighbors, in this case, will be the mean value of the 5 nearest neighbors.

In [ ]:
from sklearn.impute import KNNImputer

# Columns with missing values
null_columns = df.columns[df.isnull().any()].tolist()

# Fill Null values
imputer = KNNImputer(n_neighbors=5)
df_imp = pd.DataFrame(imputer.fit_transform(df[null_columns]), columns=null_columns)
df = df.fillna(df_imp)

This way, instead of filling all the values with the mean or the median, which is considered a simple approach, we reduce the risk of biasing the clustering results.

In [ ]:
# Distribution Visualization
plt.figure(figsize=(20,35))

for i, col in enumerate(df.columns):
    ax = plt.subplot(9, 2, i+1)
    sns.kdeplot(df[col], ax=ax)
    plt.xlabel(col)
        
plt.show()

Almost every variable is very right-skewed or very left-skewed, this indicates that maybe there are some outliers, which is not something very surprising since we are working with credit card. Certainly, there will be a small portion of people that have a very high amount of money and credit limit, while the majority portion of people has more or less the same amount of money and credit limit.

<a id="feature_scaling"></a>
## Feature Scaling

(photo)

If you see the frequency columns like BALANCE_FREQUENCY, and PURCHASES_FREQUENCY, they vary between an interval of 0 and 1, 0 means 0% of frequency and 1 means 100% frequency. Comparing them with the columns BALANCE and PURCHASES, you will see that they don't have a limit to their variation, we only know that their minimum can be 0.

However, if you put these columns in the way they're right now in the cluster, it will not yield a high-quality cluster because the cluster will understand that 1.00 dollar difference in BALANCE is as significant as 1.00 percent difference in BALANCE_FREQUENCY.
 
In virtue of that, we need to put all the columns into the same scale, otherwise, will be the same thing as clustering people on their weights in kilograms and heights in meters, is a 1kg difference as significant as a 1m difference in height?
 
This is why scaling the dataset is a vital part when using clustering techniques. Therefore, we will use Normalizer to scale the dataset, but there are many ways to scale a dataset, and each one is used for a specific situation.

In [ ]:
# Feature Scaling
from sklearn.preprocessing import Normalizer

scaler = Normalizer(norm='l2')

Normalizer, unlike the other methods, works on the rows, not the columns. This seems very unintuitive, but it means that will scale each value according to its line, and not according to the values in its column.

By default, L2 normalization is applied to each observation so the values in a row have a unit norm. Unit norm with L2 means that if each element were squared and summed, the total would equal 1. Moreover, Normalizer transforms all the features into values between -1 and 1.

<a id="pca"></a>
## PCA (Principal Component Analysis)

In [ ]:
# Reduce dimensions
from sklearn.decomposition import PCA

pca = PCA(n_components=2, random_state=1234)

PCA is a method used in unsupervised machine learning (such as clustering) that reduces high-dimension data to smaller dimensions while preserving as much information as possible. Using PCA before applying clustering algorithm reduces dimensions, data noise, and decreases computation cost.

In this article, the number of features will be reduced to 2 dimensions so that the clustering results can be visualized.


                                                                        ...
To organize better these two preprocessing steps we will embed them into a unique step with Pipeline:

In [ ]:
# Preprocessing
from sklearn.pipeline import Pipeline

# Pipeline
pipe_normalizedscaled_pca = Pipeline([('scaling', scaler), ('pca', pca)])

The dataset will be in this way after the preprocessing:

In [ ]:
# Transformed dataset
pd.DataFrame(
    pipe_normalizedscaled_pca.fit_transform(df),
    columns=['x', 'y']
)

<a id="visualization"></a>
## Cluster Visualization with Plotly

Once done the preprocessing, the only thing that remains is the modeling and visualization. And to do this I will create a function, so we can use it whenever we want.

In [ ]:
# Libs
from sklearn.cluster import KMeans
import plotly.express as px
import seaborn as sns

# Function
def Visualize_Cluster(df, pipeline, n_clusters):
    '''
    Display a scatter plot cluster after transforming the data and using it to fit KMeans Cluster 
    
        Parameters:
                df (pandas.core.frame.DataFrame): Dataframe that will be used in the Pipeline and train the KMeans Cluster
                pipeline (sklearn.pipeline.Pipeline): Transform the Dataframe
                n_clusters (int): Number of clusters that the KMeans Cluster will have
        
        Returns:
                None    
    '''
    
    data = pd.DataFrame(pipeline.fit_transform(df), columns=['x', 'y'])
    
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, max_iter=300, verbose=False, random_state=1234)
    clusters = pd.DataFrame(kmeans.fit_predict(data), columns=['Cluster']) 
    
    clusters_data = pd.concat([data, clusters], axis=1)
    
    fig = px.scatter(clusters_data, x='x', y='y', color='Cluster')
    fig.show()

First, we import the KMeans library for the clustering and the Plotly library for the cluster visualization. Then we write the function that has as parameters: the dataset for training the KMeans model, the pipeline for preprocessing, and the number of clusters in KMeans.

Internally, the function will transform the dataset according to the steps in the Pipeline, assuming that this transformation will only return two columns, which will be named "x", and "y". Create the KMeans Cluster, and inside this object, it will be passed the number of clusters previously specified in the parameters of the function as an argument, so we can visualize how many clusters we want. And in the end, we will plot the scatter graph of the clustering, each color will be a cluster.

Now, let's visualize our clustering with 5 clusters.

In [ ]:
Visualize_Cluster(df, pipe_normalizedscaled_pca, n_clusters=5)

Remember: almost all attributes of the KMeans model in the function are with the default values, the only attribute that we can change is the number of clusters of the clustering. Therefore, we can make the clustering even better by changing the parameters of the KMeans model.

Let's see how would be our clustering with 10 clusters.

In [ ]:
Visualize_Cluster(df, pipe_normalizedscaled_pca, n_clusters=10)

<a id="conclu"></a>
## Conclusion

In this first part, we started the project learning some concepts and understanding how clustering works. In part two of the series, we will give continuity to the project by learning the most common metrics used to validate a cluster and how to use these metrics to find the ideal number of clusters for this dataset.

In virtue of the content being about metrics, we will have to enter into math equations, so I will explain many things in mathematical terms (what people usually dislike, but the subject being studied requires that be this way), and I will assume that you have at least a basic algebra knowledge.

<a id="part_two"></a>
<h2><b> Part Two </b></h2>

---

#### Coming soon...

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

pipe_normalizer = Pipeline([('normalization', Normalizer())])

In [ ]:
data = pipe_normalizer.fit_transform(df)
data = pd.DataFrame(data, columns=df.columns)

def cluster_algorithm(n_clusters, dataset):
    kmean = KMeans(n_clusters=n_clusters, random_state=1234)
    labels = kmean.fit_predict(dataset)
    
    si = silhouette_score(dataset, labels, metric='euclidean', random_state=1234)
    db = davies_bouldin_score(dataset, labels)
    ch = calinski_harabasz_score(dataset, labels)
    
    return si, db, ch


n_clusters, si_list, db_list, ch_list = list(), list(), list(), list()

for i in range(2, 101):
    si, db, ch = cluster_algorithm(i, data)
    n_clusters.append(int(i))
    si_list.append(si)
    db_list.append(db)
    ch_list.append(ch)

metrics = pd.DataFrame(np.column_stack((n_clusters, si_list, db_list, ch_list)), columns=['N° Clusters', 'silhouette', 'davies_bouldin', 'calinski_harabasz'])

In [ ]:
import plotly.express as px
# Silhouette
fig = px.line(metrics, x='N° Clusters', y='silhouette', title='Silhouette')
fig.show()

In [ ]:
fig = px.line(metrics, x='N° Clusters', y='davies_bouldin', title='Davies Bouldin')
fig.show()

In [ ]:
fig = px.line(metrics, x='N° Clusters', y='calinski_harabasz', title='Calinski Harabasz')
fig.show()

In [ ]:
metrics[metrics['N° Clusters'] <= 8][['N° Clusters', 'silhouette']].sort_values(by='silhouette', ascending=False)

metrics[metrics['N° Clusters'] <= 8][['N° Clusters', 'davies_bouldin']].sort_values(by='davies_bouldin', ascending=True)

metrics[metrics['N° Clusters'] <= 8][['N° Clusters', 'calinski_harabasz']].sort_values(by='calinski_harabasz', ascending=False)

# n_cluster = 5
metrics[metrics['N° Clusters'] == 5].reset_index(drop=True)

In [ ]:
dataset = np.random.rand(8950, 16)

si, db, ch = cluster_algorithm(5, dataset)

pd.DataFrame(data=[[5, si, db, ch]], columns=['N° Clusters', 'silhouette', 'davies_bouldin', 'calinsk_harabasz'])

In [ ]:
print('Metrics of the cluster with a normal dataset:')
display(metrics[metrics['N° Clusters'] == 5].reset_index(drop=True))
print('Metrics of the cluster with a random dataset:')
display(pd.DataFrame(data=[[5, si, db, ch]], columns=['N° Clusters', 'silhouette', 'davies_bouldin', 'calinsk_harabasz']))

In [ ]:
splited_df = np.array_split(df, 5)

section, si_list, db_list, ch_list = list(), list(), list(), list()

for i in range(5):
    si, sb, ch = cluster_algorithm(5, splited_df[i])
    section.append(i+1)
    si_list.append(si)
    db_list.append(sb)
    ch_list.append(ch)


pd.DataFrame(np.column_stack([section, si_list, db_list, ch_list]), columns=['Section N°', 'silhouette', 'davies_bouldin', 'calinski_harabasz'])

In [ ]:
# data = pd.DataFrame(pipe_normalizer.fit_transform(df), columns=df.columns)
    
kmeans = KMeans(n_clusters=5, n_init=10, max_iter=300, verbose=False, random_state=1234)
labels = pd.DataFrame(kmeans.fit_predict(data), columns=['CLUSTER']) 
    
clusters_data = pd.concat([df, labels], axis=1)
clusters_data

In [ ]:
clusters_data['CLUSTER'].value_counts()

In [ ]:
fig = px.scatter(clusters_data, x='PURCHASES', y='PAYMENTS', color='CLUSTER')
fig.show()

In [ ]:
fig = px.scatter(clusters_data, x='PURCHASES', y='CASH_ADVANCE', color='CLUSTER')
fig.show()

In [ ]:
sns.pairplot(clusters_data, hue='CLUSTER', palette='bright')

In [ ]:
clusters_data.groupby(by='CLUSTER').describe()

In [ ]:
centroids = kmeans.cluster_centers_
centroids

In [ ]:
for i in range(centroids.shape[1]):
    print(f'{clusters_data.columns[i]}:', '{:.4f}'.format(centroids[:, i].var()))

In [ ]:
centroids_list = list()

for i in range(centroids.shape[1]):
    centroids_list.append(centroids[:, i].var())


centroids_var = pd.DataFrame(centroids_list, index=clusters_data.columns[:-1], columns=['Variance of the Centroids'])
centroids_var

In [ ]:
significant_centroids_var = centroids_var[centroids_var['Variance of the Centroids'] > 0.009]
significant_centroids_var

In [ ]:
from scipy import stats

stats.probplot(clusters_data['BALANCE'], plot=plt)
plt.show()

The dataset is a usage of account during the last 6 months so it means that at the start of these 6 months, lets say the amount in account is VALUE
where: VALUE = PURCHASE + BALANCE
so PURCHASE is the amount of money the account spent
and the BALANCE is the money left in VALUE

In [ ]:
balance = significant_centroids_var.index[0]

clusters_data[[balance, 'CLUSTER']].groupby(by='CLUSTER').describe()

In [ ]:
purchases = significant_centroids_var.index[1]

clusters_data[[purchases, 'CLUSTER']].groupby(by='CLUSTER').describe()

In [ ]:
cash_advance = significant_centroids_var.index[2]

clusters_data[[cash_advance, 'CLUSTER']].groupby(by='CLUSTER').describe()

In [ ]:
credit_limit = significant_centroids_var.index[3]

clusters_data[[credit_limit, 'CLUSTER']].groupby(by='CLUSTER').describe()

In [ ]:
payments = significant_centroids_var.index[4]

clusters_data[[payments, 'CLUSTER']].groupby(by='CLUSTER').describe()

In [ ]:
clusters_data[['PRC_FULL_PAYMENT', 'CLUSTER']].groupby(by='CLUSTER').describe()

In [ ]:
minimum_payments = significant_centroids_var.index[5]

clusters_data[[minimu_payments, 'CLUSTER']].groupby(by='CLUSTER').describe()

In [ ]:
clusters_data['CLUSTER'].value_counts()